In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("/home/zhaocq/桌面/tensorflow/mnist/raw/",one_hot=True)
sess = tf.InteractiveSession()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#权重和偏置的初始化函数
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)#给权重增加一些随机噪声打破完全对称
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)#给偏置增加小的正值用来避免死亡节点；
    return tf.Variable(initial)

In [3]:
#卷积层和池化层定义创建函数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1],padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [4]:
#定义输入的placeholder, x为特征，y_为真实的label
x = tf.placeholder(tf.float32,[None, 784])
y_=tf.placeholder(tf.float32,[None,10])
x_image = tf.reshape(x,[-1,28,28,1])#将2D图像由1*784转为28*28结构，第一个参数表示样本数量不固定，最后一个参数表示图像通道数

In [5]:
#构建第一个卷积层
W_conv1 = weight_variable([5,5,1,32])#[5,5,1,32]表示卷积核尺寸5*5,1通道，32个不同卷积核
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [6]:
W_conv2 = weight_variable([5,5,32,64])#[5,5,1,32]表示卷积核尺寸5*5,32通道，64个不同卷积核
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [7]:
#对第二个卷积层输出进行变形。将其转换为1D的向量
W_fc1= weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
W_fc2= weight_variable([1024,10])
b_fc2=bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

In [8]:
#损失函数定义
cross_entropy = tf.reduce_mean(-tf.reduce_mean(y_*tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [9]:
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))#tf.argmax即找出预测结果中概率最大的一个
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#tf.cast数据类型转换，bool转为float32

In [10]:
tf.global_variables_initializer().run()
for i in range(3000):
    batch = mnist.train.next_batch(100)
    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0],y_:batch[1],keep_prob:0.5}) #keep_prob训练时通常小于1，测试时为1
        print("step %d,training accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={x: batch[0],y_:batch[1],keep_prob:0.5})
print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob: 1.0}))#feed_dict给placeholder创建的tensor赋值

step 0,training accuracy 0.16
step 1000,training accuracy 0.95
step 2000,training accuracy 0.97
test accuracy 0.9847
